In [1]:
import nesmdb
import pickle
import os
import inspect
import tempfile
import subprocess
import numpy as np

In [2]:
def openScore (fileName, fileFolder = 'databases/nesmdb24_seprsco/train/'):
    with open(fileFolder+fileName, 'rb') as f:
        rate, nsamps, seprsco = pickle.load(f)
    return rate, nsamps, seprsco

In [3]:
def openScoreCSV (fileName, fileFolder = 'databases/nesmdb24_seprsco/train/'):
    seprsco = np.genfromtxt(fileFolder+fileName, delimiter=',')
    rate = 24.
    nsamps = 44100.*seprsco.shape[0]
    return rate, nsamps, seprsco

In [4]:
def convertVGM(folderRead, folderWrite=None):
    if folderWrite is None:
        folderWrite=folderRead 
    directory = os.path.dirname(folderWrite)
    if not os.path.exists(directory):
        os.mkdir(directory)  
    i = 0
    for fileName in os.listdir(folderRead):
        seprsco = openScore(fileName, folderRead)
        exprsco = nesmdb.score.seprsco_to_exprsco(seprsco)
        rawsco = nesmdb.score.exprsco_to_rawsco(exprsco)
        ndf = nesmdb.score.rawsco_to_ndf(rawsco)
        ndr = nesmdb.vgm.ndf_to_ndr(ndf)
        vgm = nesmdb.vgm.ndr_to_vgm(ndr)
        
        with open(os.path.join(folderWrite, fileName.split(".")[0]+".vgm"), "w") as vf:
            vf.write(vgm)
        subprocess.call(["vgmplay/VGMPlay.exe", os.path.abspath(os.path.join(folderWrite, fileName.split(".")[0]+".vgm"))])
        i +=1

In [5]:
def convertVGMCSV(folderRead, folderWrite=None):
    if folderWrite is None:
        folderWrite=folderRead  
    directory = os.path.dirname(folderWrite)
    if not os.path.exists(directory):
        os.mkdir(directory) 
    i = 0
    for fileName in os.listdir(folderRead):
        seprsco = openScoreCSV(fileName, folderRead)
        exprsco = nesmdb.score.seprsco_to_exprsco(seprsco)
        rawsco = nesmdb.score.exprsco_to_rawsco(exprsco)
        ndf = nesmdb.score.rawsco_to_ndf(rawsco)
        ndr = nesmdb.vgm.ndf_to_ndr(ndf)
        vgm = nesmdb.vgm.ndr_to_vgm(ndr)
        
        with open(os.path.join(folderWrite, fileName.split(".")[0]+".vgm"), "w") as vf:
            vf.write(vgm)
        subprocess.call(["vgmplay/VGMPlay.exe", os.path.abspath(os.path.join(folderWrite, fileName.split(".")[0]+".vgm"))])
            
        i +=1

In [6]:
convertVGMCSV('saved4CSV/', 'saved4VGMCSV/')

In [7]:
convertVGM('saved4/', 'saved4VGM/')